In [1]:
import json

In [2]:
import pymongo 
from pymongo import MongoClient

In [3]:
# connect to database
connection = MongoClient('localhost', 27017)

db = connection.twitter


In [4]:

# handle to names collection
tweets = db.tweets


tweets_data_path = '../data/tweets.txt'

tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets.insert(tweet)
    except:
        continue



/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:12: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
